# Análisis de Clustering en Datos preprocesados con FFT
### Objetivo
Implementar análisis de clustering en el dominio de frecuencia para verificar si existe separación evidente entre las clases de daño (N1, N2, N3) usando pares sincronizados de señales (S2, S1).

### Contexto del Proyecto
#### Estado Actual
- Datos raw: 34 especímenes, cada uno con 1-3 mediciones (pares S1/S2)
- Etiquetas: nivel_damage.csv mapea especímenes → {N1: 18, N2: 12, N3: 4}

#### Concepto Clave
- **1 datapoint = 1 par (S2, S1)** de una medición sincronizada
- El daño se manifiesta en la **relación entre S2 y S1**, no en señales individuales
- S2 = excitación basal, S1 = respuesta estructural filtrada por el aislador

In [1]:
import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# Configuración de plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

print("📦 Librerías cargadas correctamente")

📦 Librerías cargadas correctamente


## 📊 Paso 1: Cargar Pares de Señales (S2, S1)

En este paso cargamos las señales raw de vibración de cada espécimen como **pares sincronizados** (S2, S1).

### ¿Por qué emparejar S2 y S1?

En Structural Health Monitoring (SHM), un aislador sísmico funciona como un **sistema dinámico input-output**:

```
S2 (base) → [AISLADOR] → S1 (superior)
```

- **S2 (Sótano 2)**: Mide la vibración en la base del aislador (excitación)
- **S1 (Sótano 1)**: Mide la vibración sobre el aislador (respuesta filtrada)

**El daño NO se detecta mirando S1 o S2 por separado**, sino en **cómo cambia la relación entre ellos**:
- **Aislador sano**: Atenúa ciertas frecuencias (S1 < S2 en bandas específicas)
- **Aislador dañado**: Amplifica anómalamente, cambian frecuencias dominantes, aparecen armónicos

Por eso tratamos cada medición como **UN solo datapoint** = par (S2, S1).

In [2]:
# Importar función de utilidades
sys.path.append('../utils')
from clustering_utils import load_paired_signals_for_clustering

# Configurar rutas
SIGNALS_DIR = '../../data/Signals_Raw/'
LABELS_CSV = '../../data/nivel_damage.csv'

print("🔧 Configuración:")
print(f"   📂 Directorio de señales: {SIGNALS_DIR}")
print(f"   📋 Archivo de etiquetas: {LABELS_CSV}")
print(f"   🎯 Especímenes a cargar: Solo base (A1, A2, ... sin -2, -3)")

🔧 Configuración:
   📂 Directorio de señales: ../../data/Signals_Raw/
   📋 Archivo de etiquetas: ../../data/nivel_damage.csv
   🎯 Especímenes a cargar: Solo base (A1, A2, ... sin -2, -3)


In [3]:
# Cargar pares de señales (S2, S1)
paired_data = load_paired_signals_for_clustering(
    signals_dir=SIGNALS_DIR,
    labels_csv=LABELS_CSV,
    base_specimens_only=True,  # Solo especímenes base (1 medición por aislador)
    target_length=60000,       # Estandarizar a 60,000 muestras (10 min @ 100Hz)
    verbose=True               # Mostrar progreso detallado
)

print(f"\n✅ Pares cargados exitosamente: {len(paired_data)}")

📋 PASO 1: Cargando etiquetas...
   ✓ Cargadas etiquetas para 34 especímenes

📂 PASO 2: Escaneando directorio ../../data/Signals_Raw/...
   ✓ Filtrado a 14 especímenes base (sin variantes -2, -3)

🔄 PASO 3: Cargando pares (S2, S1)...
   ✓ A1: 60,000 muestras → 60,000 (sin cambios) | N1 | Tipo B
   ✓ A10: 60,000 muestras → 60,000 (sin cambios) | N2 | Tipo C
   ✓ A11: 60,000 muestras → 60,000 (sin cambios) | N2 | Tipo C
   ✓ A12: 60,000 muestras → 60,000 (sin cambios) | N1 | Tipo C
   ✓ A13: 60,000 muestras → 60,000 (sin cambios) | N1 | Tipo C
   ✓ A14: 60,000 muestras → 60,000 (sin cambios) | N1 | Tipo B
   ✓ A2: 60,000 muestras → 60,000 (sin cambios) | N1 | Tipo B
   ✓ A3: 58,699 muestras → 60,000 (padded) | N1 | Tipo C
   ✓ A4: 60,000 muestras → 60,000 (sin cambios) | N1 | Tipo B
   ✓ A5: 59,899 muestras → 60,000 (padded) | N3 | Tipo A
   ✓ A6: 60,000 muestras → 60,000 (sin cambios) | N3 | Tipo A
   ✓ A7: 60,000 muestras → 60,000 (sin cambios) | N1 | Tipo C
   ✓ A8: 60,000 muestras → 6

### 🔍 Inspección de los Datos Cargados

Verifiquemos la estructura de los datos cargados y confirmemos que todo está correcto.

In [ ]:
# Inspeccionar el primer par cargado
print("📋 ESTRUCTURA DE UN PAR (ejemplo con el primer espécimen):")
print("=" * 60)

first_pair = paired_data[0]

print(f"🆔 Specimen ID: {first_pair['specimen_id']}")
print(f"🏷️  Nivel de daño: {first_pair['nivel_dano']}")
print(f"🔧 Tipo de aislador: {first_pair['tipo']}")
print(f"📏 Longitud original: {first_pair['original_length']:,} muestras")
print(f"\n📊 Señal S2 (Sótano 2 - Base):")
print(f"   Shape: {first_pair['signal_S2'].shape}")
print(f"   Ejes: [N-S, E-W, U-D]")
print(f"   Min: [{first_pair['signal_S2'][:, 0].min():.6f}, {first_pair['signal_S2'][:, 1].min():.6f}, {first_pair['signal_S2'][:, 2].min():.6f}]")
print(f"   Max: [{first_pair['signal_S2'][:, 0].max():.6f}, {first_pair['signal_S2'][:, 1].max():.6f}, {first_pair['signal_S2'][:, 2].max():.6f}]")

print(f"\n📊 Señal S1 (Sótano 1 - Superior):")
print(f"   Shape: {first_pair['signal_S1'].shape}")
print(f"   Ejes: [N-S, E-W, U-D]")
print(f"   Min: [{first_pair['signal_S1'][:, 0].min():.6f}, {first_pair['signal_S1'][:, 1].min():.6f}, {first_pair['signal_S1'][:, 2].min():.6f}]")
print(f"   Max: [{first_pair['signal_S1'][:, 0].max():.6f}, {first_pair['signal_S1'][:, 1].max():.6f}, {first_pair['signal_S1'][:, 2].max():.6f}]")

print("=" * 60)

In [ ]:
# Crear DataFrame resumen de todos los pares cargados
summary_df = pd.DataFrame([
    {
        'Specimen_ID': pair['specimen_id'],
        'Nivel_Dano': pair['nivel_dano'],
        'Tipo': pair['tipo'],
        'Long_Original': pair['original_length'],
        'Long_Estandarizada': pair['signal_S2'].shape[0],
        'S2_RMS_NS': np.sqrt(np.mean(pair['signal_S2'][:, 0]**2)),
        'S1_RMS_NS': np.sqrt(np.mean(pair['signal_S1'][:, 0]**2))
    }
    for pair in paired_data
])

print("\n📊 RESUMEN DE TODOS LOS PARES CARGADOS:")
print(summary_df.to_string(index=False))

print(f"\n✅ PASO 1 COMPLETADO")
print(f"   • {len(paired_data)} pares (S2, S1) cargados y estandarizados")
print(f"   • Distribución: {summary_df['Nivel_Dano'].value_counts().to_dict()}")
print(f"   • Todas las señales estandarizadas a 60,000 muestras")